# Basic visualization of time series data

In [ ]:
pip install python-dotenv

In [ ]:
pip install plotly
!pip install nbformat --upgrade

In [ ]:
from dotenv import load_dotenv
load_dotenv()  # This loads the variables from .env
# Import WRDS Module: Start by importing the WRDS module in your Python script.
import wrds
import pandas as pd
import os
import numpy as np
import yfinance as yf

In [ ]:
# Get your apis or usernames
wrds_username = os.getenv('wrds_username')

In [ ]:
# Connect to database 
db = wrds.Connection(wrds_username=wrds_username)

In [ ]:
def get_stock_data(db, ticker, start_date, end_date):
    """
    Fetches adjusted stock data for a given ticker and date range from WRDS CRSP database.

    Parameters:
    ticker (str): The stock ticker symbol.
    start_date (str): The start date for the data in 'YYYY-MM-DD' format.
    end_date (str): The end date for the data in 'YYYY-MM-DD' format.

    Returns:
    pandas.DataFrame: A DataFrame containing the adjusted stock data.
    """

    # Map ticker to PERMNO
    permno_query = f"""
    SELECT permno
    FROM crsp.stocknames
    WHERE ticker = '{ticker}'
    """
    permno_result = db.raw_sql(permno_query)
    if permno_result.empty:
        raise ValueError(f"No PERMNO found for ticker {ticker}")

    # Convert the Pandas Series to a list and then to a string
    permno_list = permno_result.permno.drop_duplicates().tolist()
    formatted_permnos = ', '.join([str(permno) for permno in permno_list])

    # Query for stock data using PERMNO
    stock_data_query = f"""
    SELECT date, permno, prc, cfacpr
    FROM crsp.dsf
    WHERE permno IN ({formatted_permnos})
    AND date BETWEEN '{start_date}' AND '{end_date}'
    """
    stock_data = db.raw_sql(stock_data_query)
    
    # Calculate the adjusted close price
    stock_data['adj_close'] = stock_data['prc'] / stock_data['cfacpr']  # Note on this line below

    # Drop unnecessary columns
    stock_data = stock_data.drop(columns=["permno", "prc", "cfacpr"])

    # Convert 'date' column to datetime
    stock_data['date'] = pd.to_datetime(stock_data['date'])

    # Set 'date' column as the index
    stock_data.set_index('date', inplace=True)

    # Rename 'adj_close' column to 'AdjClose_{ticker}'
    stock_data.rename(columns={'adj_close': f'AdjClose_{ticker}'}, inplace=True)

    return stock_data


## Note on line on above code: Adjusted Stock Prices in CRSP Database

In the CRSP database provided by WRDS (Wharton Research Data Services), the adjusted price is typically available, but it might not be directly named `adjprc`. The CRSP database often includes a column for daily stock prices (`prc`) and a factor to adjust these prices (`cfacpr`) for splits and dividends. The adjusted price can be calculated by dividing the price by this factor.

### Key Points:
- **Daily Stock Prices (`prc`)**: This column represents the raw closing prices of stocks on a daily basis.
- **Adjustment Factor (`cfacpr`)**: This factor is used to adjust the raw stock prices for corporate actions like stock splits and dividends.
- **Calculation of Adjusted Prices**: The adjusted price can be obtained by dividing the `prc` value by the `cfacpr` value. This adjusted price reflects the stock's value accounting for corporate actions.

By using these columns from the CRSP database, we can accurately analyze the stock prices over time, taking into account any corporate actions that might affect the stock's value.


In [ ]:
# Example usage
ticker = 'MSFT'  # Replace with desired ticker
start_date = "2018-01-01"  # Replace with desired start date
end_date = "2020-12-31"  # Replace with desired end date
stock_data = get_stock_data(db, ticker, start_date, end_date)
stock_data.head()

In [ ]:
# Download same data using yahoo finance 
df = yf.download(ticker,
                 start=start_date, 
                 end=end_date,
                 auto_adjust = False, 
                 progress=False)

df.head()

The differences in adjusted close prices obtained from the WRDS CRSP database and Yahoo Finance, accessed via `yfinance`, stem from a variety of factors:

- **Data Source and Methodology**: Both WRDS CRSP and Yahoo Finance may employ distinct methodologies for calculating adjusted close prices, which are designed to account for corporate actions such as dividends, stock splits, and rights offerings. These adjustments can vary significantly between data providers.

- **Dividend Reinvestment Assumption**: Some sources might assume dividend reinvestment, which can elevate the adjusted price, while others do not. This leads to discrepancies in the adjusted close prices between different sources.

- **Timing of Adjustments**: The timing of these adjustments also plays a crucial role, as different sources may update their data at varying speeds in response to corporate actions, resulting in temporary discrepancies.

- **Historical Data Revisions**: Financial data providers often revise historical data to enhance accuracy, leading to potential differences in historical prices across sources.

- **Data Quality and Errors**: The inherent possibility of data errors or quality issues in any financial database can also contribute to these differences, underscoring the importance of considering these factors when analyzing and comparing financial data from multiple sources.

--- 
# How should I choose which data source to use for my project? 

Deciding which data source to trust between WRDS CRSP, Yahoo Finance (via yfinance), among others depends on several factors, including the specific requirements of your analysis, the level of detail and accuracy needed, and the context in which you're using the data. 

The short answer to such question would be "it depends". if your work demands the highest standards of data accuracy and depth (such as in academic research), WRDS CRSP is generally the better choice. For more general purposes, Yahoo Finance provides a good balance of accessibility and reliability. In your project, you may need to argue your choice on database selection.

In [ ]:
import pandas as pd

def get_compustat_data(db, ticker, start_date, end_date):
    """
    Fetches adjusted stock data for a given ticker and date range from WRDS Compustat database.

    Parameters:
    ticker (str): The stock ticker symbol.
    start_date (str): The start date for the data in 'YYYY-MM-DD' format.
    end_date (str): The end date for the data in 'YYYY-MM-DD' format.

    Returns:
    pandas.DataFrame: A DataFrame containing the adjusted stock data.
    """

    # Query for stock data using ticker
    stock_data_query = f"""
    SELECT a.datadate, a.gvkey, a.tic, a.conm, a.at, b.prccm, b.cshoq
    FROM comp.funda a
    INNER JOIN comp.secm b
    ON a.gvkey = b.gvkey
    AND a.iid = b.iid
    AND a.datadate = b.datadate
    WHERE a.tic = '{ticker}'
    AND a.datadate >= '{start_date}'
    AND a.datadate <= '{end_date}'
    AND a.datafmt = 'STD'
    AND a.consol = 'C'
    AND a.indfmt = 'INDL'
    """
    stock_data = db.raw_sql(stock_data_query, date_cols=['datadate'])

    # Assuming 'prccm' is the monthly closing price, you might need to adjust it
    # If there's an adjustment factor available in the dataset, use it here
    # For example: stock_data['adj_close'] = stock_data['prccm'] / stock_data['adjustment_factor']

    # Process the DataFrame as needed (e.g., set index, rename columns)
    stock_data.set_index('datadate', inplace=True)
    stock_data.rename(columns={'prccm': 'Close_Price'}, inplace=True)

    return stock_data


In [ ]:
df1=get_compustat_data(db, ticker, start_date, end_date)
df1

In [ ]:
# Get annual data 
stock_data_query = f"""
SELECT a.datadate, a.gvkey, a.tic, a.conm, a.at, b.prccm, b.cshoq
FROM comp.funda a
INNER JOIN comp.secm b
ON a.gvkey = b.gvkey
AND a.iid = b.iid
AND a.datadate = b.datadate
WHERE a.tic = '{ticker}'
AND a.datadate >= '{start_date}'
AND a.datadate <= '{end_date}'
AND a.datafmt = 'STD'
AND a.consol = 'C'
AND a.indfmt = 'INDL'
"""
stock_data_1 = db.raw_sql(stock_data_query, date_cols=['datadate'])
stock_data_1

The above code is good for yearly data but what about monthly? 

To get the monthly data you can use the [Compustat Security Monthly (SECM)](https://wrds-www.wharton.upenn.edu/pages/support/support-articles/compustat/north-america/closed-end-funds/) 

In [ ]:
# Extract first 5 obs from comp.secm in order to investigate it

secm = db.get_table(library='comp', table='secm', obs=5)
secm

In [ ]:
# See the columns and do your own research on these columns
secm.columns

In [ ]:
# Get Monthly data 
stock_data_query = f"""
SELECT b.datadate, b.gvkey, b.tic, b.prccm, b.cshoq
FROM comp.secm b
WHERE b.tic = '{ticker}'
AND b.datadate >= '{start_date}'
AND b.datadate <= '{end_date}'
"""

stock_data_2 = db.raw_sql(stock_data_query, date_cols=['datadate'])
stock_data_2.head()

# prccm is Index Price - Close Monthly read more about it here https://wrds-www.wharton.upenn.edu/data-dictionary/comp_global/g_idx_mth/#row-prccm

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# feel free to modify, for example, change the context to "notebook"
sns.set_theme(context="talk", style="whitegrid", 
              palette="colorblind", color_codes=True, 
              rc={"figure.figsize": [12, 8]})

In [ ]:
stock_data.plot(title="MSFT stock in 2020")

sns.despine()
plt.tight_layout()

In [ ]:
stock_data.plot(title="MSFT stock in 2020", backend="plotly")

---
# Visualizing seasonal patterns

Your book speaks about unemployment rate in the US, let us try to get unemployment rate for females in the UK from the Office of National Statistics (ONS)
Get unemployment rate of females over 16 from office of national statistics and download it to your current working directory (CWD). You can reach it [here](https://www.ons.gov.uk/employmentandlabourmarket/peoplenotinwork/unemployment/timeseries/mgsz/lms)


In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
data = pd.read_csv('series-051223.csv')

# Display the first few rows of the DataFrame to verify
data.head()


# Metadata

CSV file you downloaded contains metadata and the actual data mixed together, which is common with datasets from official sources like the Office of National Statistics. 

To properly load and use this data in Python, we'll need to adjust the way we read the file with pandas to skip the initial rows of metadata and start reading from where the actual data begins.

In [ ]:


# Adjust the number in skiprows to match where your actual data starts
# For example, if your data starts at row 10, set skiprows=9
data = pd.read_csv('series-051223.csv', skiprows=8, names=['Year', 'Female Unemployment Rate'])
data.head()


In [ ]:
data.tail()

# Further Examination

Examing the data, it seems that we have an issue were we have various category of year. Some rows have quarters and other have month so we need to do the following

1. Separate the Data: Use string manipulation and conditional logic to separate the data into the three categories.
2. Create Separate DataFrames: Based on the separation, create the three DataFrames with the appropriate columns.
3. Plot the Data: Use pandas with Plotly backend to plot the data for the years 2014-2019.

In [ ]:
# Initialize empty DataFrames
df2_year = pd.DataFrame(columns=['Year', 'Female Unemployment Rate'])
df2_quarter = pd.DataFrame(columns=['Year', 'Quarter', 'Female Unemployment Rate'])
df2_month = pd.DataFrame(columns=['Year', 'Month', 'Female Unemployment Rate'])

In [ ]:
# Function to check if a string represents an integer (year)
def is_integer(n):
    try:
        int(n)
        return True
    except ValueError:
        return False

# Lists to store data temporarily
year_data = []
quarter_data = []
month_data = []

# Iterate over the rows and separate the data
for index, row in data.iterrows():
    time_split = row['Year'].split()  # Replace 'Date' with the actual column name

    if len(time_split) == 1 and is_integer(time_split[0]):  # Year only
        year_data.append({'Year': int(time_split[0]), 'Female Unemployment Rate': row['Female Unemployment Rate']})
    elif len(time_split) == 2:
        if time_split[1].startswith('Q'):  # Year and Quarter
            quarter_data.append({'Year': int(time_split[0]), 'Quarter': time_split[1], 'Female Unemployment Rate': row['Female Unemployment Rate']})
        else:  # Year and Month
            month_data.append({'Year': int(time_split[0]), 'Month': time_split[1], 'Female Unemployment Rate': row['Female Unemployment Rate']})

# Convert lists to DataFrames
df2_year = pd.concat([df2_year, pd.DataFrame(year_data)], ignore_index=True)
df2_quarter = pd.concat([df2_quarter, pd.DataFrame(quarter_data)], ignore_index=True)
df2_month = pd.concat([df2_month, pd.DataFrame(month_data)], ignore_index=True)


In [ ]:
df2_year.tail()

In [ ]:
df2_quarter.tail()

In [ ]:
# Ensure you have Plotly installed: pip install plotly
pd.options.plotting.backend = "plotly"

# Filter data for years 2014-2019
df2_year_filtered = df2_year[(df2_year['Year'] >= 2014) & (df2_year['Year'] <= 2019)]

# Plotting
fig = df2_year_filtered.plot(x='Year', y='Female Unemployment Rate', title="Female Unemployment rate in years 2014-2019")
fig.show()


In [ ]:

# Filter data for years 2014-2019
df2_quarter_filtered = df2_quarter[(df2_quarter['Year'] >= 2014) & (df2_quarter['Year'] <= 2019)]

# Combine Year and Quarter for plotting
df2_quarter_filtered['Year-Quarter'] = df2_quarter_filtered['Year'].astype(str) + ' ' + df2_quarter_filtered['Quarter']

# Filter data for years 2014-2019
df2_quarter_filtered = df2_quarter_filtered[(df2_quarter_filtered['Year'] >= 2014) & (df2_quarter_filtered['Year'] <= 2019)]

# Plotting
fig = df2_quarter_filtered.plot(x='Year-Quarter', y='Female Unemployment Rate', title="Female Unemployment rate (Quarterly) in years 2014-2019")
fig.show()

# Plotting monthly data while only showing the year on the x-axis 

This can be a bit tricky, especially if you want to maintain the granularity of the monthly data in the plot. 

One approach is to plot all the monthly data points but format the x-axis labels to only show the year. This can be done by creating a new column that represents the year and using it for the x-axis, while still plotting the monthly data.


## Step 1: Prepare the Data

First, ensure that the 'Year' and 'Month' columns are in the correct format. Then, create a new column for plotting purposes.

In [ ]:
import pandas as pd

# Convert 'Year' to string if it's not already
df2_month['Year'] = df2_month['Year'].astype(str)

# Create a new column 'Year-Month' for plotting
df2_month['Year-Month'] = pd.to_datetime(df2_month['Year'] + ' ' + df2_month['Month'])

# Filter data for years 2014-2019
df2_month_filtered = df2_month[(df2_month['Year-Month'].dt.year >= 2014) & (df2_month['Year-Month'].dt.year <= 2019)]

## Step 2: Plotting with Custom X-axis

Now, plot the data using Plotly, and format the x-axis to only show the year.

In [ ]:
# Ensure you have Plotly installed: pip install plotly
pd.options.plotting.backend = "plotly"

# Plotting
fig = df2_month_filtered.plot(x='Year-Month', y='Female Unemployment Rate', title="UK Female Unemployment Rate (Monthly) in Years 2014-2019")

# Customize x-axis to show only the year
fig.update_xaxes(
    tickvals=[pd.to_datetime(f'{y}-01-01') for y in range(2014, 2020)],  # Set ticks at the start of each year
    ticktext=[str(y) for y in range(2014, 2020)],  # Label ticks with the year only
)

# Get current figure height and increase it by 20%
current_height = fig.layout.height or 400  # Default height is 400 if not set
new_height = current_height * 2  # Increase height by 20%


# Update figure layout with the new height and title properties
fig.update_layout(
    height=new_height,
    title={
        'text': "UK Female Unemployment Rate (Monthly) in Years 2014-2019",
        'y':0.93,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 24}
    }
)


fig.show()

# Monthly data for each year from 2014 to 2019 on a single figure

To compare to your main textbook, we will plot the data in one year (i.e. 12 month each year) from 2014 till 2019, to get some understanding of trend and seasonality

In [ ]:
import plotly.express as px


## Step 1: Prepare the Data
First, ensure that your DataFrame is in the correct format. You'll need to convert the 'Month' column to a format that allows for easy plotting on a common scale (January to December) for each year.

In [ ]:
# Convert 'Year' to integer and 'Month' to datetime
df2_month['Year'] = df2_month['Year'].astype(int)
df2_month['Month'] = pd.to_datetime(df2_month['Month'], format='%b').dt.month

# Filter data for years 2014-2019
df2_month_filtered = df2_month[(df2_month['Year'] >= 2014) & (df2_month['Year'] <= 2019)]

## Step 2: Plotting with Plotly Express
Now, plot the data using Plotly Express, ensuring each year is represented as a separate line:

In [ ]:
# Plotting with Plotly Express
fig = px.line(df2_month_filtered, x='Year-Month', y='Female Unemployment Rate', color='Year', 
              title="Female Unemployment Rate (Monthly) in Years 2014-2019")

# Customize the layout
fig.update_layout(
    xaxis_title='Month',
    yaxis_title='Female Unemployment Rate',
    legend_title='Year'
)

fig.show()


In [ ]:
import plotly.express as px

# Plotting with Plotly Express
fig = px.line(df2_month_filtered, x='Month', y='Female Unemployment Rate', color='Year', 
              title="Female Unemployment Rate (Monthly) in Years 2014-2019")

# Customize the layout
fig.update_layout(
    xaxis_title='Month',
    yaxis_title='Female Unemployment Rate',
    legend_title='Year',
    xaxis = dict(tickmode = 'array',
                 tickvals = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
                 ticktext = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
)

# Get current figure height and increase it by 20%
current_height = fig.layout.height or 400  # Default height is 400 if not set
new_height = current_height * 2  # Increase height by 20%


# Update figure layout with the new height and title properties
fig.update_layout(
    height=new_height,
    title={
        'text': "UK Female Unemployment Rate (Monthly) in Years 2014-2019",
        'y':0.93,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 24}
    }
)


fig.show()


In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import pandas as pd

def month_plot(data, ylabel):
    # Pivot the data for plotting
    data_pivot = data.pivot("Year", "Month", "Female Unemployment Rate")
    
    # Create a heatmap
    sns.heatmap(data_pivot, annot=True, fmt=".1f", linewidths=.5, cmap="coolwarm")
    plt.ylabel(ylabel)


In [ ]:
# Convert 'Year' to integer and 'Month' to datetime
df2_month['Year'] = df2_month['Year'].astype(int)
# If the 'Month' column contains numerical month representations (1, 2, 3, etc.)
df2_month['Month'] = df2_month['Month'].astype(int)  # Convert to integer if not already

# Filter data for years 2014-2019
df2_month_filtered = df2_month[(df2_month['Year'] >= 2014) & (df2_month['Year'] <= 2019)]


In [ ]:
# Plotting
month_plot(df2_month_filtered, ylabel="Unemployment rate (%)")
plt.title("Unemployment rate - Month plot")

sns.despine()
plt.tight_layout()
plt.show()


In [ ]:


Creating interactive visualizations
Creating a candlestick chart